# Efficient Yelp API Calls (Core)
- Coire Gavin-Hanner
- 9/27/22

# Objective:
- Use Yelp API to search a city for a particular type of cuisine
    - Here I am searching in Seattle for Ethiopian food
- Extract all of the results and compile them into one dataframe

# Imports and Definitions

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [4]:
def create_json_file(JSON_FILE,  replace_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if replace_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Make the new file
            folder = os.path.dirname(JSON_FILE)
            if len(folder)>0:
            # create the folder
                os.makedirs(folder,exist_ok=True)
            ## Save empty list to start the json file
            with open(JSON_FILE,'w') as f:
                json.dump([],f) 
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

# Load API Credentials

In [2]:
# Load API Credentials
with open('/Users/coire/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

### Define Search

In [44]:
# set our API call parameters 
LOCATION = 'Seattle'
CATEGORIES = 'ethiopian'
TERM = 'restaurant'

# Create Results File

In [45]:
JSON_FILE = "Data/results_in_progress_Seattle_vegan.json"

In [59]:
create_json_file(JSON_FILE, replace_if_exists=True)

[!] Data/results_in_progress_Seattle_vegan.json already exists. Deleting previous file...


# Load Previous Results and set offset/pages

In [60]:
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)

n_results = len(previous_results)

In [61]:
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [62]:
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                                categories=CATEGORIES,
                                offset=n_results)

In [63]:
total_results = results['total']

In [64]:
results_per_page = len(results['businesses'])

In [65]:
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

2

# Calls

In [66]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM,
                                    categories=CATEGORIES,
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/2 [00:00<?, ?it/s]

# Extract results to Dataframe

In [67]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,aMHOtT5WI4inmIV8kavmIw,adey-abeba-ethiopian-restaurant-seattle,Adey Abeba Ethiopian Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/nQPg8h...,False,https://www.yelp.com/biz/adey-abeba-ethiopian-...,188,"[{'alias': 'ethiopian', 'title': 'Ethiopian'}]",4.5,"{'latitude': 47.61279550818363, 'longitude': -...",[delivery],$$,"{'address1': '2123 E Union St', 'address2': ''...",12063221617,(206) 322-1617,2757.012161
1,9bZtK6ca9AXFlv3FX9sRPQ,adulis-eritrean-and-ethiopian-restaurant-seattle,Adulis Eritrean and Ethiopian Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/ZTUHU_...,False,https://www.yelp.com/biz/adulis-eritrean-and-e...,6,"[{'alias': 'eritrean', 'title': 'Eritrean'}, {...",4.0,"{'latitude': 47.70562, 'longitude': -122.35578}",[],NaN,"{'address1': '10515 Greenwood Ave N', 'address...",12069205973,(206) 920-5973,9049.703501
2,mVe-ouCiPHElEXaAyfv1IA,café-soleil-seattle-2,Café Soleil,https://s3-media4.fl.yelpcdn.com/bphoto/oTv0zX...,False,https://www.yelp.com/biz/caf%C3%A9-soleil-seat...,87,"[{'alias': 'ethiopian', 'title': 'Ethiopian'},...",4.5,"{'latitude': 47.61311, 'longitude': -122.28909}",[delivery],$$,"{'address1': '1400 34th Ave', 'address2': '', ...",12063251126,(206) 325-1126,3741.392104
3,uiuHZbOXHYXmV9ahQXNIZA,shewa-ber-bar-and-restaurant-seattle,Shewa-Ber Bar & Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/QiGeQ4...,False,https://www.yelp.com/biz/shewa-ber-bar-and-res...,19,"[{'alias': 'ethiopian', 'title': 'Ethiopian'},...",5.0,"{'latitude': 47.5993552129099, 'longitude': -1...",[],NaN,"{'address1': '1801 S Jackson St', 'address2': ...",14252239000,(425) 223-9000,3535.354311
4,-ZvCzmfQltMYgiGb_niOnQ,cafe-selam-seattle,Cafe Selam,https://s3-media3.fl.yelpcdn.com/bphoto/QbT0A1...,False,https://www.yelp.com/biz/cafe-selam-seattle?ad...,330,"[{'alias': 'ethiopian', 'title': 'Ethiopian'}]",4.5,"{'latitude': 47.60779, 'longitude': -122.29707}",[delivery],$$,"{'address1': '2715 E Cherry St', 'address2': '...",12063280404,(206) 328-0404,3482.878449


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
31,Nq2QBSPq40WZgUzXD93u7w,blue-nile-seattle-2,Blue Nile,https://s3-media2.fl.yelpcdn.com/bphoto/x4METy...,False,https://www.yelp.com/biz/blue-nile-seattle-2?a...,56,"[{'alias': 'ethiopian', 'title': 'Ethiopian'}]",4.0,"{'latitude': 47.53196, 'longitude': -122.2706}","[delivery, pickup]",$,"{'address1': '7821 Rainier Ave S', 'address2':...",12063208501,(206) 320-8501,11474.303088
32,5IQopbLST4fHUitX4W0V1w,kaffa-coffee-and-wine-bar-seattle-2,Kaffa Coffee & Wine Bar,https://s3-media1.fl.yelpcdn.com/bphoto/BgA9tv...,False,https://www.yelp.com/biz/kaffa-coffee-and-wine...,65,"[{'alias': 'ethiopian', 'title': 'Ethiopian'},...",4.5,"{'latitude': 47.5298805, 'longitude': -122.269...","[delivery, pickup]",$$,"{'address1': '8136 Rainier Ave S', 'address2':...",12064533558,(206) 453-3558,11721.818344
33,aXM_hRDLrvgX87_jUUQy5g,paradise-restaurant-seattle,Paradise Restaurant,,False,https://www.yelp.com/biz/paradise-restaurant-s...,4,"[{'alias': 'ethiopian', 'title': 'Ethiopian'}]",4.5,"{'latitude': 47.5373712765657, 'longitude': -1...",[delivery],NaN,"{'address1': '7250 Rainier Ave S', 'address2':...",12066594106,(206) 659-4106,10944.137819
34,M_REjnKHx8HQk-5DnL-C0A,monona-cafe-and-restaurant-seattle,Monona Cafe and Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/hzGNCF...,False,https://www.yelp.com/biz/monona-cafe-and-resta...,11,"[{'alias': 'ethiopian', 'title': 'Ethiopian'},...",4.0,"{'latitude': 47.5408671047097, 'longitude': -1...","[delivery, pickup]",$,"{'address1': '6754 Martin Luther King Jr S', '...",12067660607,(206) 766-0607,10198.404587
35,PJHQqeUaiVQg96cyCnexJA,ethio-market-seattle,Ethio Market,https://s3-media2.fl.yelpcdn.com/bphoto/nYG4QN...,False,https://www.yelp.com/biz/ethio-market-seattle?...,2,"[{'alias': 'ethiopian', 'title': 'Ethiopian'},...",3.0,"{'latitude': 47.5270146, 'longitude': -122.26955}",[],NaN,"{'address1': '8432 Rainier Ave S', 'address2':...",12067258389,(206) 725-8389,12008.755122


## List of Restaurants

In [68]:
final_df['alias']

0               adey-abeba-ethiopian-restaurant-seattle
1      adulis-eritrean-and-ethiopian-restaurant-seattle
2                                 café-soleil-seattle-2
3                  shewa-ber-bar-and-restaurant-seattle
4                                    cafe-selam-seattle
5                                     chef-cafe-seattle
6                                    ras-dashen-seattle
7                             alem-restaurant-seattle-2
8                    delish-ethiopian-cuisine-seattle-2
9                                     cafe-ibex-seattle
10                                queen-sheba-seattle-3
11          east-african-imports-and-restaurant-seattle
12                                  jebena-cafe-seattle
13                 zagol-ethiopian-restaurant-seattle-3
14                                  sunset-cafe-seattle
15                                        zobel-seattle
16                                kezira-cafe-seattle-3
17                                 habesha-cafe-